# Code generation

**pyecsca** can generate C implementations of ECC crypto for several microprocessor targets, which
are defined by the `Platform` enum.

In [3]:
from pyecsca.codegen.common import Platform


Platform.names()

['HOST', 'XMEGA', 'STM32F0', 'STM32F3']

To generate an implementation we need an actual configuration which the implementation should
implement, which is stored inside the `Configuration` and `DeviceConfiguration` classes. 

In [8]:
from typing import get_args
from pyecsca.codegen.common import DeviceConfiguration
from dataclasses import fields

for field in fields(DeviceConfiguration):
	name = field.name
	tp = field.type
	doc = tp.__doc__
	if get_args(field.type):
		doc = get_args(field.type)[0].__doc__
	if tp == bool:
		doc = ""
	print(name, tp)
	print("   ", doc)
	if hasattr(tp, "names"):
		for enum_name in tp.names():
			print("       ", enum_name)
	print()

model <class 'pyecsca.ec.model.CurveModel'>
    A model(form) of an elliptic curve.

coords <class 'pyecsca.ec.coordinates.CoordinateModel'>
    A coordinate system for a particular model(form) of an elliptic curve.

formulas typing.Set[pyecsca.ec.formula.Formula]
    A formula operating on points.

scalarmult <class 'pyecsca.ec.mult.ScalarMultiplier'>
    
    A scalar multiplication algorithm.

    :param short_circuit: Whether the use of formulas will be guarded by short-circuit on inputs
                          of the point at infinity.
    :param formulas: Formulas this instance will use.
    

hash_type <enum 'HashType'>
    Hash algorithm used in ECDH and ECDSA.
        NONE
        SHA1
        SHA224
        SHA256
        SHA384
        SHA512

mod_rand <enum 'RandomMod'>
    Method of sampling a uniform integer modulo order.
        SAMPLE
        REDUCE

mult <enum 'Multiplication'>
    Base multiplication algorithm to use.
        TOOM_COOK
        KARATSUBA
        COMB

The `DeviceConfiguration` class contains a few additional attributes apart from those
in the `Configuration` class: `platform`, `keygen`, `ecdh` and `ecdsa`.

The `platform` attribute defines for which target the implementation
should be built. The other boolean attributes specify whether particular
functionality should be implemented and enabled in the implementation.

## Generating

We will first create a `DeviceConfiguration`, which we will then generate and build.

In [13]:
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.configuration import *

platform = Platform.HOST
hash_type = HashType.SHA1
mod_rand = RandomMod.REDUCE
mult = Multiplication.BASE
sqr = Squaring.BASE
red = Reduction.BASE

model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
add = coords.formulas["add-1998-cmo"]
dbl = coords.formulas["dbl-1998-cmo"]
scl = coords.formulas["z"]
formulas = [add, dbl, scl]
scalarmult = LTRMultiplier(add, dbl, scl)

config = DeviceConfiguration(model, coords, formulas, scalarmult, 
							 hash_type, mod_rand, mult, sqr, red,
							 platform, True, True, True)

config

DeviceConfiguration(model=ShortWeierstrassModel(), coords=EFDCoordinateModel("projective" on short Weierstrass curves), formulas=[AdditionEFDFormula(add-1998-cmo for EFDCoordinateModel("projective" on short Weierstrass curves)), DoublingEFDFormula(dbl-1998-cmo for EFDCoordinateModel("projective" on short Weierstrass curves)), ScalingEFDFormula(z for EFDCoordinateModel("projective" on short Weierstrass curves))], scalarmult=<pyecsca.ec.mult.LTRMultiplier object at 0x7fb1da4ddd00>, hash_type=HASH_SHA1, mod_rand=MOD_RAND_REDUCE, mult=MUL_BASE, sqr=SQR_BASE, red=RED_BASE, platform=HOST, keygen=True, ecdh=True, ecdsa=True)

Now we can render the configuration, which will generate the source files into a
randomly created temporary directory, and return the path to the directory as
well as names of the elf and hex files which will be built in that directory.

In [12]:
from pyecsca.codegen.builder import render

directory, elf_name, hex_name = render(config)

print(directory)

/tmp/tmpszimqnob


## Building

When we have the implementation rendered, we can build it using make.

In [15]:
from subprocess import run

res = run(["make"], cwd=directory, capture_output=True)
print(res.stdout.decode())

rm -f -- pyecsca-codegen-HOST.hex
rm -f -- pyecsca-codegen-HOST.eep
rm -f -- pyecsca-codegen-HOST.cof
rm -f -- pyecsca-codegen-HOST.elf
rm -f -- pyecsca-codegen-HOST.map
rm -f -- pyecsca-codegen-HOST.sym
rm -f -- pyecsca-codegen-HOST.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- main.s bn/bn.s asn1/asn1.s hash/hash.s prng/prng.s gen/point.s gen/curve.s gen/mult.s simpleserial.s uart.s host_hal.s
rm -f -- main.d bn/bn.d asn1/asn1.d hash/hash.d prng/prng.d gen/point.d gen/curve.d gen/mult.d simpleserial.d uart.d host_hal.d
rm -f -- main.i bn/bn.i asn1/asn1.i hash/hash.i prng/prng.i gen/point.i gen/curve.i gen/mult.i simpleserial.i uart.i host_hal.i
.
-------- begin --------
gcc (GCC) 9.2.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: main.c
gcc -c  -I. -gdwarf-2 -DHASH=HASH_SHA1 -DMOD_RAND=MOD_RAND_REDUCE -DSS

Now the files `elf_name` and `hex_name` in the directory contain the ELF file and HEX file built.


In [17]:
res = run(["file", elf_name], cwd=directory, capture_output=True)
print(res.stdout.decode())

res = run(["file", hex_name], cwd=directory, capture_output=True)
print(res.stdout.decode())

pyecsca-codegen-HOST.elf: ELF 64-bit LSB pie executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, BuildID[sha1]=b120c193e0d3f480bec86e817c0b08e3f9ed74c7, for GNU/Linux 3.2.0, with debug_info, not stripped

pyecsca-codegen-HOST.hex: ASCII text, with CRLF line terminators

